## [GTSRB - German Traffic Sign Recognition Benchmark](https://www.kaggle.com/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/notebooks) 

In [ ]:
!pip install visualkeras

In [ ]:
import visualkeras
import pandas as pd
df=pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv',
               usecols=['ClassId','Path','Width','Height']).rename(columns={"ClassId": "label",
                                                                            "Path": "path"})

test=pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv',usecols=['ClassId','Path']).rename(columns={"ClassId": "label"})


print(df.head())
print('\n-----------No. of Classes-----------')
print(df.label.nunique())

Images file path follows following file structure
`main_directory/
...class_a/
......a_image_1.jpg
......a_image_2.jpg
...class_b/
......b_image_1.jpg
......b_image_2.jpg
`

Where classes are total 43, so we can use tensorflow `image_dataset_from_directory` function utility.

In [ ]:
import tensorflow as tf
# Training path for images to grab
train_img_path='../input/gtsrb-german-traffic-sign/Train'
# Define Std batch size of 32
batch_size = 32

# As images size vary from 26 X 26 to 173 X 173, use std img size
img_height = 26
img_width = 26


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_img_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_img_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Define the model architecture

In [ ]:
# Importing all essential lib
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
import tensorflow.keras.layers.experimental.preprocessing as preprocessing

num_classes=43

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.experimental.preprocessing.RandomContrast(0.6),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


visualkeras.layered_view(model)

In [ ]:
model.summary()

Train for 10 epochs

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
    batch_size=32, 
)

Accuracy graph

In [ ]:
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

Validation Accuracy

In [ ]:
print(f"Accuracy: {history_frame['val_accuracy'][9]*100} %")
# model.save('/kaggle/working/model_traffic_data1.h5')